# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_file_df = ("../output_data/weather_globalcities.csv")
weather_df = pd.read_csv(weather_file_df)
weather_df.head()


,Unnamed: 0,city,country,lat,lon,temp,temp_max,humidity,wind_speed,cloudiness,date
0,0,Yangi Marg`ilon,UZ,40.4272,71.7189,26.60,26.60,63,4.61,0,1611511156
1,1,Jertih,MY,5.7336,102.4897,77.00,77.00,83,3.44,20,1611511156
2,2,Albany,US,42.6001,-73.9662,20.21,21.99,35,3.00,0,1611510891
3,3,Mangaratiba,BR,-22.9597,-44.0406,95.00,95.00,36,9.22,20,1611511157
4,4,Hobart,AU,-42.8794,147.3294,64.85,68.00,49,10.36,88,1611511157


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["lat", "lon"]].astype(float)


# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Narrow down above dataframe to include my ideal weather.
ideal_weather_df = weather_df

ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["temp_max"] < 75) & (ideal_weather_df["temp_max"] > 50)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["cloudiness"] < 70) & (ideal_weather_df["cloudiness"] > 25)]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["wind_speed"] < 10]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df.head()

,Unnamed: 0,city,country,lat,lon,temp,temp_max,humidity,wind_speed,cloudiness,date
6,6,Launceston,AU,-41.4500,147.1667,57.88,60.01,72,3.44,41,1611511157
61,61,Misratah,LY,32.3754,15.0925,63.91,63.91,46,6.49,65,1611511168
81,81,Mount Gambier,AU,-37.8333,140.7667,71.60,71.60,49,8.05,60,1611511172
141,141,Victoria Point,AU,-27.5833,153.3000,71.73,73.99,80,1.99,39,1611511184
163,163,Debre Sīna,ET,9.8500,39.7667,50.43,50.43,55,3.33,55,1611511188


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#POTATO - START HERE

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
